In [1]:
!pip install mlxtend 
import pandas as pd
import numpy as np
import time
import mlxtend
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

In [2]:
#read data
data='交易資料集.xlsx'
df_read = pd.read_excel(data)
df = df_read.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157396 entries, 0 to 157395
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   INVOICE_NO    155484 non-null  object        
 1   CUST_ID       157396 non-null  int64         
 2   ITEM_ID       157396 non-null  int64         
 3   ITEM_NO       157373 non-null  object        
 4   PRODUCT_TYPE  155135 non-null  object        
 5   TRX_DATE      155481 non-null  datetime64[ns]
 6   QUANTITY      157396 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 8.4+ MB


In [3]:
df.head()

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016-07-26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016-07-11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016-07-27,3000
3,A20160700174,999999999,3434776,IHLP1616ABER2R2M11,PEMCO,2016-07-29,0
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016-07-06,12000


In [4]:
#preprocessing
df = df[df['QUANTITY'] > 0]
df

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016-07-26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016-07-11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016-07-27,3000
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016-07-06,12000
6,CX47347899,10228,3420352,TMP103AYFFR,LINEAR IC,2016-07-25,3000
...,...,...,...,...,...,...,...
157389,216071271,2100,14691206,"74HCT365D,653",LOGIC IC,2016-07-06,7030
157390,216075314,2197,3249493,"74HC04D,653",LOGIC IC,2016-07-21,347
157391,216072287,2579,133619,"74LVC2G17GW,125",LOGIC IC,2016-07-11,1644
157392,216077517,43262,15231095,MT41K256M16TW-107:P TRAY,MEMORY_EMBEDED,2016-07-28,300000


In [5]:
df2 = df.drop(['CUST_ID','ITEM_NO','PRODUCT_TYPE','TRX_DATE','QUANTITY'],axis=1)
df2

,INVOICE_NO,ITEM_ID
0,CX47348203,3217532
1,CX47346522,3326781
2,CX47348534,740487
4,CX47346184,70072
6,CX47347899,3420352
...,...,...
157389,216071271,14691206
157390,216075314,3249493
157391,216072287,133619
157392,216077517,15231095


In [6]:
df2.nunique()

INVOICE_NO    36498
ITEM_ID       23710
dtype: int64

In [7]:
#transaction
itemset = []
transaction = {}
for i in df2.index:
    if df2['ITEM_ID'][i] not in itemset:
        itemset.append(df2['ITEM_ID'][i])
    if str(df2['INVOICE_NO'][i]) in transaction:
        transaction[str(df2['INVOICE_NO'][i])].append(str(df2['ITEM_ID'][i]))
    else:
        transaction[str(df2['INVOICE_NO'][i])] = [str(df2['ITEM_ID'][i])]

dataset = []
for i in transaction:
    dataset.append(transaction[i])

In [8]:
te = TransactionEncoder()
teAry = te.fit(dataset).transform(dataset)
dfInput = pd.DataFrame(teAry, columns =te.columns_)
dfInput

,100,1000,100016,100018,10002587,100026,100030,10018587,10021587,10028588,...,995630,995631,9964587,996629,996630,9971587,9972587,9980587,999,9999587
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36493,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36494,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36495,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
#apriori find frequent itemset with min_sup=0.0008
frequent_itemset_apriori = apriori(dfInput, min_support = 0.0008, use_colnames = True)
frequent_itemset_apriori

,support,itemsets
0,0.000904,(100)
1,0.000877,(1007631)
2,0.001480,(111064)
3,0.001014,(12560591)
4,0.002028,(125741)
...,...,...
495,0.000849,"(88411, 88622, 88623)"
496,0.000849,"(135493, 88411, 135492, 70509)"
497,0.000877,"(135493, 88676, 88411, 70509)"
498,0.000822,"(1698, 1697, 88411, 70509)"


In [10]:
#apriori rules with confidence = 0.7
rules_apriori = association_rules(frequent_itemset_apriori, metric="confidence", min_threshold=0.7)
rules_apriori

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(135492),(135493),0.001260,0.001671,0.000986,0.782609,468.256593,0.000984,4.592312,0.999124
1,(135492),(70509),0.001260,0.008137,0.001041,0.826087,101.516908,0.001031,5.703210,0.991399
2,(135492),(88411),0.001260,0.004576,0.001069,0.847826,185.293153,0.001063,6.541360,0.995858
3,(135492),(88676),0.001260,0.001507,0.000904,0.717391,476.060870,0.000902,3.533129,0.999159
4,(135493),(70509),0.001671,0.008137,0.001233,0.737705,90.655738,0.001219,3.781476,0.990625
...,...,...,...,...,...,...,...,...,...,...
122,"(1698, 88411, 1708)",(70509),0.000822,0.008137,0.000822,1.000000,122.888889,0.000815,inf,0.992679
123,"(1698, 70509, 1708)",(88411),0.000959,0.004576,0.000822,0.857143,187.329341,0.000818,6.967971,0.995617
124,"(1708, 88411, 70509)",(1698),0.000932,0.003534,0.000822,0.882353,249.644323,0.000819,8.469957,0.996923
125,"(1698, 1708)","(88411, 70509)",0.001069,0.002082,0.000822,0.769231,369.412955,0.000820,4.324310,0.998360


In [11]:
#apriori write file
rules_apriori['antecedents'] = rules_apriori['antecedents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules_apriori['consequents'] = rules_apriori['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
writer = pd.ExcelWriter('association_rules_apriori.xlsx')
asso_rules_apriori = rules_apriori[['antecedents','consequents','antecedent support','consequent support','support','confidence']]
asso_rules_apriori.to_excel(writer, sheet_name='Apriori', index = False)
writer.close()

In [12]:
# apriori read file
df_apriori_rule = pd.read_excel('association_rules_apriori.xlsx')
df_apriori_rule.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence
0,135492,135493,0.001260,0.001671,0.000986,0.782609
1,135492,70509,0.001260,0.008137,0.001041,0.826087
2,135492,88411,0.001260,0.004576,0.001069,0.847826
3,135492,88676,0.001260,0.001507,0.000904,0.717391
4,135493,70509,0.001671,0.008137,0.001233,0.737705


In [13]:
#apriori recommend production
def get_consequents(antecedents, min_confidence):
    consequents = []
    for  i in df_apriori_rule['antecedents'].index:
        if antecedents in df_apriori_rule['antecedents'][i] and  df_apriori_rule['confidence'][i]>min_confidence :
            consequents.append(df_apriori_rule['consequents'][i])
    return consequents

#min_sup = 0.0008
recommend =get_consequents('88411',0.7)
print('min_sup=0.0008 item:88411 number of recommend rules product:')
print(len(recommend))
print(recommend)

min_sup=0.0008 item:88411 number of recommend rules product:
38
['135493', '135492', '70509', '135492', '88676', '135493', '135493', '70509', '135493', '88676', '70509', '70509', '1698', '70509', '70509', '70509', '70509', '70509', '70509', '70509', '70509', '70509', '70509', '70509', '88623', '88622', '70509', '135492', '135493', '70509, 135493', '70509', '88676', '135493', '70509, 135493', '70509', '70509', '1698', '1698, 70509']


In [14]:
# different support same confidence =0.7 and write file
min_sup = [0.0008, 0.001, 0.0012]
rules = []
time_consumption = []
count =1
writer = pd.ExcelWriter('association_rules_apriori.xlsx')
for i in min_sup:
    start_time = time.time()
    frequent_itemset_apriori = apriori(dfInput, min_support = i, use_colnames = True)
    rules_apriori = association_rules(frequent_itemset_apriori, metric="confidence", min_threshold=0.7)
    time_consumption.append(time.time()-start_time)
    rules.append(len(rules_apriori['antecedents']))
    rules_apriori['antecedents'] = rules_apriori['antecedents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    rules_apriori['consequents'] = rules_apriori['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    asso_rules_apriori = rules_apriori[['antecedents','consequents','antecedent support','consequent support','support','confidence']]
    asso_rules_apriori.to_excel(writer, sheet_name='Apriori_min_sup'+str(count), index = False)
    count+=1
writer.close()
for j in range(len(min_sup)):
    print('min support: ',min_sup[j],' rule counts: ', rules[j], ' time consumption: %0.3f seconds'%time_consumption[j])

min support:  0.0008  rule counts:  127  time consumption: 30.173 seconds
min support:  0.001  rule counts:  23  time consumption: 14.913 seconds
min support:  0.0012  rule counts:  9  time consumption: 7.791 seconds


In [15]:
# different confidence same support =0.001
min_conf = [0.5, 0.6, 0.7]
rules = []
time_consumption = []
for i in min_conf:
    start_time = time.time()
    frequent_itemset_apriori = apriori(dfInput, min_support = 0.001, use_colnames = True)
    rules_apriori = association_rules(frequent_itemset_apriori, metric = "confidence", min_threshold = i)
    time_consumption.append(time.time()-start_time)
    rules.append(len(rules_apriori['antecedents']))
for j in range(len(min_conf)):
    print('min confidence: ',min_conf[j],' rule counts: ', rules[j], ' time consumption: %0.3f seconds'%time_consumption[j])

min confidence:  0.5  rule counts:  57  time consumption: 14.936 seconds
min confidence:  0.6  rule counts:  39  time consumption: 14.551 seconds
min confidence:  0.7  rule counts:  23  time consumption: 14.584 seconds


In [16]:
# read file and number of recoomend products
for i in range(1,4):
    sheet = 'Apriori_min_sup'+str(i)
    df_apriori_rule = pd.read_excel('association_rules_apriori.xlsx',sheet)
    print('----min_sup ='+str(min_sup[i-1])+'----')
    print(len(get_consequents('88411',0.7)))

----min_sup =0.0008----
38
----min_sup =0.001----
4
----min_sup =0.0012----
2


In [17]:
#fpgrowth find frequent itemset with min_sup=0.001
frequent_itemset_fp = fpgrowth(dfInput, min_support = 0.0008, use_colnames = True)
frequent_itemset_fp

,support,itemsets
0,0.004740,(23004)
1,0.001891,(87118)
2,0.000959,(126991)
3,0.003178,(70072)
4,0.002986,(149674)
...,...,...
495,0.000877,"(15280990, 14790248)"
496,0.001425,"(14782215, 14790248)"
497,0.001096,"(14929876, 14790248)"
498,0.000822,"(15336088, 15280990)"


In [18]:
#fpgrowth rules with confidence = 0.7
rules_fp = association_rules(frequent_itemset_fp, metric="confidence", min_threshold=0.7)
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(1708, 23004)",(70509),0.001343,0.008137,0.000986,0.734694,90.285714,0.000975,3.738559,0.990254
1,"(88411, 87118)",(70509),0.000877,0.008137,0.000877,1.000000,122.888889,0.000870,inf,0.992733
2,"(87118, 70509)",(88411),0.001096,0.004576,0.000877,0.800000,174.840719,0.000872,4.977122,0.995371
3,"(1697, 88411)",(70509),0.001315,0.008137,0.001233,0.937500,115.208333,0.001222,15.869801,0.992626
4,"(1697, 70509)",(88411),0.001562,0.004576,0.001233,0.789474,172.540183,0.001226,4.728266,0.995759
...,...,...,...,...,...,...,...,...,...,...
122,(88675),(70509),0.001123,0.008137,0.000849,0.756098,92.915989,0.000840,4.066637,0.990350
123,(14876857),(14870535),0.001123,0.001096,0.001096,0.975610,890.195122,0.001095,40.955066,1.000000
124,(14870535),(14876857),0.001096,0.001123,0.001096,1.000000,890.195122,0.001095,inf,0.999973
125,(14752201),(14760944),0.001562,0.001781,0.001562,1.000000,561.507692,0.001559,inf,0.999780


In [19]:
#fpgrowth write file
rules_fp['antecedents'] = rules_fp['antecedents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules_fp['consequents'] = rules_fp['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
writer = pd.ExcelWriter('association_rules_fp.xlsx')
asso_rules_fp = rules_fp[['antecedents','consequents','antecedent support','consequent support','support','confidence']]
asso_rules_fp.to_excel(writer, sheet_name='FPgrowth', index = False)
writer.close()

In [20]:
# fpgrowth read file
df_fp_rule = pd.read_excel('association_rules_fp.xlsx')
df_fp_rule.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence
0,"1708, 23004",70509,0.001343,0.008137,0.000986,0.734694
1,"88411, 87118",70509,0.000877,0.008137,0.000877,1.000000
2,"87118, 70509",88411,0.001096,0.004576,0.000877,0.800000
3,"1697, 88411",70509,0.001315,0.008137,0.001233,0.937500
4,"1697, 70509",88411,0.001562,0.004576,0.001233,0.789474


In [21]:
#fpgrowth recommend products
def get_consequents(antecedents, min_confidence):
    consequents = []
    for  i in df_fp_rule['antecedents'].index:
        if antecedents in df_fp_rule['antecedents'][i] and  df_fp_rule['confidence'][i]>min_confidence :
            consequents.append(df_fp_rule['consequents'][i])
    return consequents
#min_sup = 0.0008
recommend =get_consequents('88411',0.7)
print('min_sup=0.0008 item:88411 number of recommend rules product:')
print(len(recommend))
print(recommend)

min_sup=0.0008 item:88411 number of recommend rules product:
38
['70509', '70509', '70509', '70509', '70509', '70509', '70509', '70509', '1698', '70509', '1698', '1698, 70509', '70509', '135493', '70509', '70509', '70509', '135493', '88676', '70509', '70509', '88676', '135493', '70509, 135493', '70509', '135493', '135492', '135493', '135492', '70509', '135493, 70509', '135492', '88676', '70509', '135493', '88623', '88622', '70509']


In [22]:
# different support same confidence =0.7 and write file
min_sup = [0.0008, 0.001, 0.0012]
rules = []
time_consumption = []
count =1
writer = pd.ExcelWriter('association_rules_fp.xlsx')
for i in min_sup:
    start_time = time.time()
    frequent_itemset_fp = fpgrowth(dfInput, min_support = i, use_colnames = True)
    rules_fp = association_rules(frequent_itemset_fp, metric="confidence", min_threshold=0.7)
    time_consumption.append(time.time()-start_time)
    rules.append(len(rules_fp['antecedents']))
    rules_fp['antecedents'] = rules_fp['antecedents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    rules_fp['consequents'] = rules_fp['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    asso_rules_fp = rules_fp[['antecedents','consequents','antecedent support','consequent support','support','confidence']]
    asso_rules_fp.to_excel(writer, sheet_name='FPgrowth_min_sup'+str(count), index = False)
    count+=1
writer.close()    
for j in range(len(min_sup)):
    print('min support: ',min_sup[j],' rule counts: ', rules[j], ' time consumption: %0.3f seconds'%time_consumption[j])

min support:  0.0008  rule counts:  127  time consumption: 1.060 seconds
min support:  0.001  rule counts:  23  time consumption: 1.047 seconds
min support:  0.0012  rule counts:  9  time consumption: 1.035 seconds


In [23]:
#different confidence same support =0.001
min_conf = [0.5, 0.6, 0.7]
rules = []
time_consumption = []
for i in min_conf:
    start_time = time.time()
    frequent_itemset_fp = fpgrowth(dfInput, min_support = 0.001, use_colnames = True)
    rules_fp = association_rules(frequent_itemset_fp, metric = "confidence", min_threshold = i)
    time_consumption.append(time.time()-start_time)
    rules.append(len(rules_fp['antecedents']))
for j in range(len(min_conf)):
    print('min confidence: ',min_conf[j],' rule counts: ', rules[j], ' time consumption: %0.3f seconds'%time_consumption[j])

min confidence:  0.5  rule counts:  57  time consumption: 1.124 seconds
min confidence:  0.6  rule counts:  39  time consumption: 1.045 seconds
min confidence:  0.7  rule counts:  23  time consumption: 1.041 seconds


In [24]:
# read file and number of recoomend products
for i in range(1,4):
    sheet = 'FPgrowth_min_sup'+str(i)
    df_fp_rule = pd.read_excel('association_rules_fp.xlsx',sheet)
    print('----min_sup ='+str(min_sup[i-1])+'----')
    print(len(get_consequents('88411',0.7)))

----min_sup =0.0008----
38
----min_sup =0.001----
4
----min_sup =0.0012----
2
